In [1]:
import re, collections 
import numpy as np
import nltk

In [30]:
def assign_numbers(vocab_list):
    '''This function assigns index numbers to words in a vocabulary.'''
    vocab_indexer = dict()
    index_value = 2
    for word in vocab_list:
        vocab_indexer[word] = index_value
        index_value = index_value+1
    return vocab_indexer

def token_to_index(tokens, token_indexer):
    """
    Function that transforms a list of tokens in a document and coverts it to a list of token indices.
    @param tokens: list of tokens from one document
    @param token_indexer: dictionary that maps each token in the vocabulary to an unique index
    """
    # Please DO NOT assign any ngram in the vocabulary to index 0
    document = []
    for token in tokens:
        try:  
            document.append(token_indexer[token])
        except:
            document.append(1)
    return document


def split_tokens_into_characters(counter):
    '''This function takes a counter (indicating the counts per token across the entire corpus) and 
    splits each token into a series of characters'''
    char_dict = {}
    old_dict = dict(counter)
    for token in old_dict.keys():
        char_form = " ".join(token)+' </w>' #add ending character
        value = old_dict[token]
        char_dict[char_form] = value 
    return char_dict

def get_stats(vocab):
    '''This function counts the number of times each pair of characters appears next to each other.
    For example the sentence "baking a cake" would return (a,k):2, (b,a):1, (k,e):1, and so on.
    '''
    pairs = collections.defaultdict(int) 
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq 
    return pairs

def merge_vocab(pair, v_in):
    '''This function changes the vocabulary so that n-grams that co-occur together frequently
    are merged'''
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)') 
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word] 
    return v_out

def convert_tokens_into_frequent_character_combinations(counter, num_merges):
    '''This function takes a Counter object (counts per token) and converts it so that 
    the tokens (keys) become split into frequent character combinations.
    For example, {'cake': 3, 'baking': 1} might become {'c ak e': 3, 'b ak ing': 1}
    
    @num_merges is the number of "rounds" for merging character blocks. For each round, the 
    pair of characters that co-occur the most are combined. 
    
    @num_merges is a very important hyperparameter.
    '''
    char_dict = split_tokens_into_characters(counter)
    for i in range(num_merges):
        pairs = get_stats(char_dict)
        best = max(pairs, key=pairs.get) 
        char_dict = merge_vocab(best, char_dict)
        print('Sample: '+ str(char_dict))
    return char_dict, pairs

def create_vocabulary_of_character_combos(char_dict):
    '''Takes the character_dictionary from above and creates an indexed vocabulary 
    for frequent character combinations.
    
    For example, {'c ak e': 3, 'b ak ing': 1} would become something like
    {'c': 1, 'b': 2, 'ak': 3, 'e': 4, 'ing': 5}. 
    
    Eventually we want to map each unigram to a series of character blocks 
    (e.g. 'baking' becomes 2 3 5)
    '''
    #First, find the set of all grams
    grams = []
    for word in char_dict.keys():
        word_grams = word.split(' ')
        grams.extend(word_grams)
    GRAMS = list(np.unique(grams)) #Delete any duplicates
    Vocab_Index = assign_numbers(GRAMS)
    return Vocab_Index

def map_unigram_to_character_indices(char_dict, Vocab_Index):
    '''This function maps each unigram token to a series of numbers. Each number corresponds to a 
    character block.
    
    For example, if Vocab_Index = {'c': 1, 'b': 2, 'ak': 3, 'e': 4, 'ing': 5}, 
    "baking" should be mapped to [2, 3, 5]
    
    
    @char_dict is the counter-like object where the keys are unigrams split into character blocks, 
    values are the number of times the unigram appears.
    '''
    unigrams = {} 
    for x in char_dict.keys():
        unigram = x.replace(' ','').replace('</w>','') #e.g. revert "b ak ing<w>" to baking
        unigrams[unigram] = x
    
    for w in unigrams.keys():
        unigrams[w] = token_to_index(unigrams[w].split(' '), Vocab_Index)
    return unigrams



In [31]:
old_cts = {'low' : 5, 'lower' : 2, 'newest':6,'widest':3} #e.g. "low" appears 5 times in corpus

In [32]:
weird_cts, pairs = convert_tokens_into_frequent_character_combinations(old_cts, 7)
weird_cts

Sample: {'l o w </w>': 5, 'w i d es t </w>': 3, 'l o w e r </w>': 2, 'n e w es t </w>': 6}
Sample: {'l o w e r </w>': 2, 'n e w est </w>': 6, 'l o w </w>': 5, 'w i d est </w>': 3}
Sample: {'n e w est</w>': 6, 'w i d est</w>': 3, 'l o w e r </w>': 2, 'l o w </w>': 5}
Sample: {'l ow e r </w>': 2, 'w i d est</w>': 3, 'n e w est</w>': 6, 'l ow </w>': 5}
Sample: {'low </w>': 5, 'w i d est</w>': 3, 'n e w est</w>': 6, 'low e r </w>': 2}
Sample: {'ne w est</w>': 6, 'w i d est</w>': 3, 'low </w>': 5, 'low e r </w>': 2}
Sample: {'new est</w>': 6, 'low </w>': 5, 'w i d est</w>': 3, 'low e r </w>': 2}


{'low </w>': 5, 'low e r </w>': 2, 'new est</w>': 6, 'w i d est</w>': 3}

In [37]:
#Just an example. If we were to do one more round, we would either merge "ne" and "w" to "new,
#or merge "w" and "est</w>" to "west</w>"
pairs

defaultdict(int,
            {('d', 'est</w>'): 3,
             ('e', 'r'): 2,
             ('i', 'd'): 3,
             ('low', '</w>'): 5,
             ('low', 'e'): 2,
             ('ne', 'w'): 6,
             ('r', '</w>'): 2,
             ('w', 'est</w>'): 6,
             ('w', 'i'): 3})

In [5]:
char_vocab = create_vocabulary_of_character_combos(weird_cts)
char_vocab

{'</w>': 2,
 'd': 3,
 'e': 4,
 'est</w>': 5,
 'i': 6,
 'low': 7,
 'new': 8,
 'r': 9,
 'w': 10}

In [6]:
map_unigram_to_character_indices(weird_cts, char_vocab)

{'low': [7, 2],
 'lower': [7, 4, 9, 2],
 'newest': [8, 5],
 'widest': [10, 6, 3, 5]}